In [9]:
import numpy as np
import math
import time

# Veri dosyasını okuma
def read_tsp_data(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    num_cities = int(lines[0].strip())
    cities = []
    for line in lines[1:]:
        x, y = map(int, line.strip().split())
        cities.append((x, y))
    return num_cities, cities

# Öklid mesafesi hesaplama
def calculate_distance(city1, city2):
    return math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Mesafe matrisini oluşturma
def create_distance_matrix(cities):
    num_cities = len(cities)
    distance_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            distance = calculate_distance(cities[i], cities[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix

# Karınca Kolonisi Optimizasyonu (ACO) algoritması
class AntColonyOptimization:
    def __init__(self, distance_matrix, num_ants, num_iterations, alpha=1, beta=2, evaporation_rate=0.5, Q=100):
        self.distance_matrix = distance_matrix
        self.num_cities = distance_matrix.shape[0]
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha  # Feromon etkisi
        self.beta = beta  # Mesafe etkisi
        self.evaporation_rate = evaporation_rate  # Feromon buharlaşma oranı
        self.Q = Q  # Feromon artışı sabiti
        self.pheromone = np.ones((self.num_cities, self.num_cities))  # Başlangıç feromon seviyesi

    def run(self):
        best_path = None
        best_path_length = float('inf')
        
        for iteration in range(self.num_iterations):
            all_paths = self.generate_all_paths()
            self.update_pheromone(all_paths)
            
            for path, length in all_paths:
                if length < best_path_length:
                    best_path = path
                    best_path_length = length
        
        # En iyi yolu başladığı düğüme geri döndür
        best_path.append(best_path[0])
        return best_path, best_path_length

    def generate_all_paths(self):
        all_paths = []
        for ant in range(self.num_ants):
            path = self.generate_path()
            path_length = self.calculate_path_length(path)
            all_paths.append((path, path_length))
        return all_paths

    def generate_path(self):
        path = []
        start_city = np.random.randint(0, self.num_cities)
        path.append(start_city)
        visited = set([start_city])

        while len(path) < self.num_cities:
            current_city = path[-1]
            next_city = self.select_next_city(current_city, visited)
            path.append(next_city)
            visited.add(next_city)
        
        return path

    def select_next_city(self, current_city, visited):
        probabilities = []
        total_prob = 0

        for next_city in range(self.num_cities):
            if next_city in visited:
                continue
            pheromone = self.pheromone[current_city][next_city]
            distance = self.distance_matrix[current_city][next_city]
            prob = (pheromone ** self.alpha) * ((1 / distance) ** self.beta)
            probabilities.append((next_city, prob))
            total_prob += prob

        probabilities = [(city, prob / total_prob) for city, prob in probabilities]
        probabilities.sort(key=lambda x: x[1], reverse=True)

        rand = np.random.rand()
        cumulative_prob = 0
        for city, prob in probabilities:
            cumulative_prob += prob
            if rand <= cumulative_prob:
                return city

    def calculate_path_length(self, path):
        length = 0
        for i in range(len(path)):
            current_city = path[i]
            next_city = path[(i + 1) % len(path)]
            length += self.distance_matrix[current_city][next_city]
        return length

    def update_pheromone(self, all_paths):
        self.pheromone *= (1 - self.evaporation_rate)  # Feromon buharlaştırma
        for path, length in all_paths:
            for i in range(len(path)):
                current_city = path[i]
                next_city = path[(i + 1) % len(path)]
                self.pheromone[current_city][next_city] += self.Q / length
                self.pheromone[next_city][current_city] += self.Q / length

# Ana fonksiyon
if __name__ == "__main__":
    file_path = "tsp_150.txt"
    num_cities, cities = read_tsp_data(file_path)
    distance_matrix = create_distance_matrix(cities)

    num_ants = 20
    num_iterations = 100
    aco = AntColonyOptimization(distance_matrix, num_ants, num_iterations)

    # Çalışma süresi ölçümü
    start_time = time.time()

    best_path, best_path_length = aco.run()

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Çıktılar
    print("\nBest Path:", best_path)
    print("Best Path Length:", best_path_length)
    print(f"Çalışma Süresi: {elapsed_time:.2f} saniye")



Best Path: [68, 21, 5, 119, 107, 24, 148, 131, 110, 88, 23, 65, 84, 16, 93, 29, 140, 118, 137, 125, 51, 59, 70, 83, 100, 135, 56, 106, 41, 98, 66, 120, 49, 61, 127, 75, 9, 101, 89, 87, 3, 34, 138, 94, 116, 129, 141, 105, 121, 28, 73, 80, 128, 58, 149, 96, 36, 82, 30, 91, 99, 14, 108, 69, 33, 22, 72, 12, 102, 113, 112, 145, 48, 146, 7, 136, 62, 37, 32, 38, 132, 50, 25, 130, 43, 79, 117, 10, 57, 124, 19, 18, 47, 15, 0, 45, 35, 144, 63, 55, 53, 26, 97, 76, 109, 142, 104, 1, 39, 4, 71, 139, 54, 31, 134, 64, 143, 42, 147, 77, 133, 115, 13, 46, 95, 114, 86, 40, 78, 67, 17, 123, 92, 103, 111, 44, 90, 81, 6, 2, 52, 8, 85, 126, 74, 20, 11, 27, 122, 60, 68]
Best Path Length: 30922.634178975517
Çalışma Süresi: 24.44 saniye
